# Revised case normalization for Hirslanden Aarau 2017

Notes:



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Klinik Aarau', '2017', 'Aarau_2017')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Klinik Aarau', year='2017', sheet='Aarau_2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 08:55:33.845 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 14 cases for Hirslanden Klinik Aarau 2017
2022-11-21 08:55:33.849 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                     string
tranchen                  string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    str

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,41206174,1176423,W,72,5,41206174,T840,T840,M200,3,I21C,[E1140],[],"[80051D, 815255]",[009031::20170106]
1,41230776,1659346,W,67,19,41230776,G001,G001,M100,3,B72C,[B370],[],[],[]
2,41281042,1823347,M,67,6,41281042,E1150,E1150,M100,3,F14B,[F101],[],[],[]
3,41386032,1051316,M,74,10,41386032,S7211,S7211,M200,3,I08C,[L8914],[],[],[]
4,41418378,22136564,M,86,8,41418378,C182,C182,M200,3,G18B,[E1191],[E1190],[],[]
5,41247420,1251372,W,63,8,41247420,M4216,M4216,M200,3,I09D,[M4196],[],[],[]
6,41477047,1536048,W,74,6,41477047,M315,M315,M100,3,I66B,[K3182],[K5581],[],[]
7,0041280551,0022114393,W,84,7,41280551,I5012,I5012,M100,3,F62C,"[F023, F338, F051]","[F03, F328]",[],[]
8,0041457913,0001193700,W,92,5,41457913,C341,C341,M100,3,E71B,[R15],[],[],[]
9,0041523346,0002096726,W,77,9,41523346,N1783,N1783,M100,3,L60D,[A099],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.dtypes

2022-11-21 08:55:39.458 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 10839 rows from the DB, for the hospital 'Hirslanden Klinik Aarau' in 2017


aimedic_id                 int64
case_id                   object
primary_diagnosis         object
secondary_diagnoses       object
primary_procedure         object
secondary_procedures      object
gender                    object
age_years                  int64
age_days                   int64
gestation_age              int64
duration_of_stay           int64
ventilation_hours          int64
grouper_admission_type    object
admission_date            object
admission_weight           int64
grouper_discharge_type    object
discharge_date            object
dtype: object

In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 08:55:39.914 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 14 cases ...
2022-11-21 08:55:40.882 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 14 cases into: 14 revisions, 141 diagnoses rows, 89 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,59690,I08B,3.66,3.66,4,2022-12-31
1,60516,I66A,1.97,1.97,4,2022-12-31
2,62943,G18A,2.79,2.79,4,2022-12-31
3,63300,F35B,2.53,2.53,4,2022-12-31
4,63756,F14A,3.47,3.47,4,2022-12-31
5,64240,F62B,1.38,1.38,4,2022-12-31
6,64470,B72A,2.65,2.65,4,2022-12-31
7,65934,I09C,2.80,2.80,3,2022-12-31
8,66233,I46A,2.66,2.66,4,2022-12-31
9,67449,F35B,2.53,2.53,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 08:55:40.901 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 14 cases into the 'Revisions' table ...
2022-11-21 08:55:42.106 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 14 cases into the 'Revisions' table
2022-11-21 08:55:42.109 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 141 rows into the 'Diagnoses' table ...
2022-11-21 08:55:44.365 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 141 rows into the 'Diagnoses' table
2022-11-21 08:55:44.367 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 89 rows into the 'Procedures' table ...
2022-11-21 08:55:46.535 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 89 rows into the 'Procedures' table
